In [1]:
table = open("../data/paperTable.tsv","r")
entries = []
for line in table:
    entries.append(line.split('\t'))
table.close()

In [2]:
abstracts = []
titleofabstract = []
yearofabstract = []
for articles in entries:
    titleofabstract.append(articles[0])
    abstracts.append(articles[1]+articles[2]+articles[3])
    yearofabstract.append(articles[4][:-1])

In [3]:
# Create a set of frequent words
stopfile = open("../data/stopwords.txt","r")
stopwords = stopfile.read().splitlines()
stopwords.append("\xc2\xa9") #This is the copyright symbol, this shows up in every abstract and should not be apart of the corpus
stopwords.extend(["\u2019","\u03bc"])

stoplist = set(stopwords)
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in abstracts]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
processed_corpus = [[token for token in text if frequency[token] > 5] for text in texts]
# processed_corpus

In [4]:
from gensim import corpora
import os, tempfile
TEMP_FOLDER = tempfile.gettempdir()

dictionary = corpora.Dictionary(processed_corpus)
dictionary.save(os.path.join(TEMP_FOLDER,'words.dict'))
print(dictionary)

Dictionary(4564 unique tokens: [u'limited', u'reductions', u'similarity', u'believed', u'proteins,']...)


In [5]:
new_doc = "limited bees important"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

corpus = [dictionary.doc2bow(text) for text in processed_corpus]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'words.dict'), corpus)
for c in corpus:
    print(c)

[(7, 1), (117, 1), (364, 1)]
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 4), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 6), (59, 1), (60, 1), (61, 1)]
[(4, 1), (6, 1), (7, 2), (11, 1), (13, 1), (26, 3), (28, 1), (33, 1), (51, 1), (59, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 4), (71, 1), (72, 1), (73, 2), (74, 2), (75, 2), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 3), (82, 1), (83, 2), (84, 1), (85, 1), (86, 4), (87, 1), (88, 1), (89, 1), (90, 1), (91, 2), (92, 1), (93, 1), (94, 7), (95, 4), (96, 1), (97, 1), (98, 1), (

In [6]:
from gensim.models import TfidfModel
from gensim import models

tfidf = TfidfModel(corpus,id2word=dictionary) 
lsi = models.LsiModel(tfidf[corpus], id2word=dictionary, num_topics=25)

In [15]:
topics_list = lsi.print_topics(25)

In [21]:
print lsi[dictionary.doc2bow(abstracts[0].split())]
print lsi.

[(0, 2.034892747017116), (1, -0.9518273827679667), (2, -0.16572999027625632), (3, -1.3026534488909363), (4, 1.8719809912137075), (5, 0.21722567725420194), (6, -1.1049959831603287), (7, 0.03407071777506324), (8, 0.14213723765147793), (9, 0.519594542034054), (10, -1.048498759301562), (11, -0.913413008977742), (12, 0.7625483738548149), (13, 0.27478242933313984), (14, 0.47330116427930735), (15, 0.25373017269076176), (16, -0.4479293185754165), (17, -0.31088967581055227), (18, -0.22963230735602946), (19, 0.9300746692713258), (20, -0.4893540673609181), (21, -1.3724067885884885), (22, 0.059035400478592695), (23, -1.4132071280119558), (24, -0.6046737684795472)]
1044


In [19]:
doc_topics, word_topics, phi_values = lsi.get_document_topics(lsi[dictionary.doc2bow(abstracts[0].split())], per_word_topics=True)

AttributeError: 'LsiModel' object has no attribute 'get_document_topics'